### Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from selenium.webdriver import Chrome
import requests
import lxml.html
import os
from glob import glob
import olefile
from tqdm import tqdm
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from kiwipiepy import Kiwi
from wordcloud import WordCloud
import time
from itertools import combinations
from sklearn.decomposition import TruncatedSVD
from gensim.models.fasttext import FastText
from gensim.matutils import Sparse2Corpus
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from sklearn.model_selection import train_test_split


plt.rc('font',family='malgun gothic')
plt.rc('axes', unicode_minus=False)
%matplotlib inline

### Web scraping

In [ ]:
# Web Scrapping with selenium
texts=[]
browser = Chrome()
for p in tqdm(range(1,118)): # 페이지 이동
    try: 
        browser.get(f'https://www.kmst.go.kr/kmst/verdict/writtenVerdict/selectWrittenVerdict.do?pageIndex={p}')
        urls = browser.find_elements_by_css_selector('dt a')
        for i in range(len(urls)-1): # 첫 게시글부터 마지막 게시글까지 이동
                urls[i].click()
                if browser.find_elements_by_css_selector('span.btn_pack.c_blue') != []: # 서버에서 게시글 들어가지 못하게 할 때
                    browser.find_elements_by_css_selector('span.btn_pack.c_blue')[0].click() # 돌아가기 버튼 클릭
                    urls[i].click() # 다시 한 번 더 게시글 진입 시도
                text = browser.find_elements_by_css_selector('dd.description')[0].text # 게시글 내용 선택
                texts.append(text) # 스크래핑
                browser.back() # 뒤로 가기
                urls = browser.find_elements_by_css_selector('dt a')
    except: # 오류 발생 시
        pass

In [ ]:
documents = pd.DataFrame({'text':texts}).reset_index() # 스크래핑한 내용 데이터프레임화

In [ ]:
documents.text = documents.text.map(lambda x: re.sub(r'\n+','\n',x)) # 엔터 여러 번 했을 경우 한 번으로 통일

In [ ]:
# function to separate morpheme & extract noun
def extract_n(text):
    kiwi = Kiwi()
    kiwi.prepare()
    morpheme = kiwi.analyze(text) # 형태소 분석
    for lemma, pos, _, _ in morpheme[0][0]: # 표제어, 품사 추출
        if pos.startswith('N'): # 품사가 명사일 경우
            yield lemma # 표제어 반환

In [ ]:
def spacing_non(word,text): # 게시글별로 키워드 입력 방식 달라 통일(띄어쓰기)
    spacing_list = []
    for i in range(len(word)):
        raw_x = word # 단어 raw 형태
        for split in combinations(word,i): # i개로 글자 나눔
            x = raw_x
            for s in split:
                x = x.replace(s,s+' ') # 글자 나눈 사이에 공백 추가
            spacing_list.append(x)
    for f in spacing_list:
        text =text.replace(f,raw_x) # 여러 경우의 수 하나로 통일
    return text

In [ ]:
def split_key(x):
    try:
        if x == '@DataType:Clob': # 이상한 형태일 경우 넘어간다.
            return
        x = re.sub(r'\n+','\n',x) # 엔터키 여러 번 실행했을 경우 한 번으로 통일
        x = x.replace('이 유','').replace('1. 사 실','').replace('?','.') # 불필요한 요소들 대체
        x = x.replace('5. 사고방지 교훈','4. 사고방지 교훈') # 오탈자 대체
        for key in ['주문','선명','선적항','선박소유자','총톤수','기관종류.출력','해양사고관련자','직명','면허의종류','사고일시','사고장소','2.원인','가.원인고찰',
         '나.사고발생원인','다.해양사고관련자의긴급피난주장에대한검토','3.해양사고관련자의행위','4.사고방지교훈']:
            x = spacing_non(key,x) # 키워드 형태 통일
        x1 = pd.Series(x.splitlines())
        if '따라서 주문과 같이 재결한다.' in x: # 내용 중 필요한 부분만 발췌
            content = x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]
                          :x1[x1.map(lambda x: x=='따라서 주문과 같이 재결한다.')].index[0]].reset_index(drop=True)
        else:
            try:
                content = x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]:].reset_index(drop=True)
            except:
                content = x1
        if x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].reset_index(drop=True)[0][:2] == '부산': # 부산일 경우
            key_idx_list = []
            keys1 = ['주문','선명','선적항','선박소유자','총톤수','기관종류.출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']
            keys2 = ['2.원인','가.원인고찰', '나.사고발생원인','다.해양사고관련자의긴급피난주장에대한검토','3.해양사고관련자의행위','4.사고방지교훈']
            keys = keys1+keys2
            for a in keys1: # 키워드 인덱스 번호 가져오기
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            key_idx_list.append(key_idx_list[-1]+2)
            key_con=[]
            for a in keys2:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            for i in range(len(key_idx_list)-1): # 키워드 없을 경우 결측값 대입
                if key_idx_list[i] == None:
                    key_con.append(None)
                elif key_idx_list[i+1] == None:
                    if len(key_idx_list) <= i+2:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:])))
                    else:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+2]])))
                else: # 문장 합치기
                    key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+1]])))
            if key_idx_list[-1] != None:
                key_con.append(' '.join(list(content[key_idx_list[-1]+1:])))
            else:
                key_con.append(None)
            df = pd.DataFrame(np.array([[x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]+1]]+key_con])
                                      ,columns=(['사건명']+keys1+['사건상세']+keys2)) # 데이터프레임으로 만들기
            return df # 이하동문
        elif x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].reset_index(drop=True)[0][:2] == '인천':
            for word in ['주문','선명','선적항','선박소유자','총톤수','기관종류출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']:
                x = x.replace(word,f'{word}'+'\n')
            x = x.replace('3. 해양사고관련자\n의 행위','3. 해양사고관련자의 행위')
            x1 = pd.Series(x.splitlines())
            content = x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]:].reset_index(drop=True)
            key_idx_list = []
            keys1 = ['주문','선명','선적항','선박소유자','총톤수','기관종류ㆍ출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']
            keys2 = ['2.원인','가.원인고찰', '나.사고발생원인','다.해양사고관련자의긴급피난주장에대한검토','3.해양사고관련자의행위','4.사고방지교훈']
            keys = keys1+keys2
            for a in keys1:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            key_idx_list.append(key_idx_list[-1]+2)
            key_con=[]
            for a in keys2:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            for i in range(len(key_idx_list)-1):
                if key_idx_list[i] == None:
                    key_con.append(None)
                elif key_idx_list[i+1] == None:
                    if len(key_idx_list) <= i+2:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:])))
                    else:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+2]])))
                else:
                    key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+1]])))
            if key_idx_list[-1] != None:
                key_con.append(' '.join(list(content[key_idx_list[-1]+1:])))
            else:
                key_con.append(None)
            df = pd.DataFrame(np.array([[x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]+1]]+key_con])
                                      ,columns=(['사건명']+keys1+['사건상세']+keys2)).rename(columns={'기관종류ㆍ출력':'기관종류.출력'})
            return df
        elif x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].reset_index(drop=True)[0][:2] == '동해':
            x = x.replace('사고 장소','사고장소')
            for word in ['주문','선명','선적항','선박소유자','총톤수','기관종류.출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']:
                x = x.replace(word,f'{word}'+'\n')
            x = x.replace('3. 해양사고관련자\n의 행위','3. 해양사고관련자의 행위')
            x1 = pd.Series(x.splitlines())
            content = x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]:].reset_index(drop=True)
            key_idx_list = []
            keys1 = ['주문','선명','선적항','선박소유자','총톤수','기관종류.출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']
            keys2 = ['2.원인','가.원인고찰', '나.사고발생원인','다.해양사고관련자의긴급피난주장에대한검토','3.해양사고관련자의행위','4.사고방지교훈']
            keys = keys1+keys2
            for a in keys1:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            key_idx_list.append(key_idx_list[-1]+2)
            key_con=[]
            for a in keys2:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            for i in range(len(key_idx_list)-1):
                if key_idx_list[i] == None:
                    key_con.append(None)
                elif key_idx_list[i+1] == None:
                    if len(key_idx_list) <= i+2:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:])))
                    else:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+2]])))
                else:
                    key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+1]])))
            if key_idx_list[-1] != None:
                key_con.append(' '.join(list(content[key_idx_list[-1]+1:])))
            else:
                key_con.append(None)
            df = pd.DataFrame(np.array([[x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]+1]]+key_con])
                                      ,columns=(['사건명']+keys1+['사건상세']+keys2))
            return df
        elif x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].reset_index(drop=True)[0][:2] == '목포':
            key_idx_list = []
            keys1 = ['주문','선명','선적항','선박소유자','총톤수','기관종류.출력','해양사고관련자','직명','면허의종류','사고일시','사고장소']
            keys2 = ['2.원인','가.원인고찰', '나.사고발생원인','다.해양사고관련자의긴급피난주장에대한검토','3.해양사고관련자의행위','4.사고방지교훈']
            keys = keys1+keys2
            for a in keys1:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            key_idx_list.append(key_idx_list[-1]+2)
            key_con=[]
            for a in keys2:
                if list(content[content == a]) == []:
                    key_idx_list.append(None)
                else:
                    key_idx_list.append(content[content == a].index[-1])
            for i in range(len(key_idx_list)-1):
                if key_idx_list[i] == None:
                    key_con.append(None)
                elif key_idx_list[i+1] == None:
                    if len(key_idx_list) <= i+2:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:])))
                    else:
                        key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+2]])))
                else:
                    key_con.append(' '.join(list(content[key_idx_list[i]+1:key_idx_list[i+1]])))
            if key_idx_list[-1] != None:
                key_con.append(' '.join(list(content[key_idx_list[-1]+1:])))
            else:
                key_con.append(None)
            df = pd.DataFrame(np.array([[x1.iloc[x1[x1.map(lambda x: x.endswith('호') and x.startswith('해심',2,4))].index[0]+1]]+key_con])
                                      ,columns=(['사건명']+keys1+['사건상세']+keys2))
            return df
    except:
        pass

In [ ]:
documents_keys = documents.text.map(split_key) # 재결서 데이터프레임화

In [ ]:
written_judges = pd.concat(list(documents_keys),axis=0).reset_index(drop=True) # 재결서 데이터프레임 하나로 모두 합치기

In [ ]:
# 결측값 처리 (키워드에 잘 못 들어간 요소들 처리)
written_judges.loc[written_judges['선박소유자'].map(lambda x: '톤 수' in x),'총톤수'] = written_judges[written_judges['선박소유자'].map(lambda x: '톤 수' in x)]['선박소유자'].map(lambda x: x.split('톤 수')).map(lambda x:x[1])
written_judges.loc[written_judges['선박소유자'].map(lambda x: '톤 수' in x),'선박소유자'] = written_judges[written_judges['선박소유자'].map(lambda x: '톤 수' in x)]['선박소유자'].map(lambda x: x.split('톤 수')).map(lambda x:x[0])

In [ ]:
written_judges.loc[written_judges['선박소유자'].map(lambda x: '총톤수' in x),'총톤수'] = written_judges[written_judges['선박소유자'].map(lambda x: '총톤수' in x)]['선박소유자'].map(lambda x: x.split('총톤수')).map(lambda x:x[1])
written_judges.loc[written_judges['선박소유자'].map(lambda x: '총톤수' in x),'선박소유자'] = written_judges[written_judges['선박소유자'].map(lambda x: '총톤수' in x)]['선박소유자'].map(lambda x: x.split('총톤수')).map(lambda x:x[0])

In [ ]:
written_judges.loc[written_judges['총톤수'].map(lambda x: '기관종류' in x),'기관종류.출력'] = written_judges.loc[written_judges['총톤수'].map(lambda x: '기관종류' in x),'총톤수'].map(
    lambda x: x.split('기관종류')[1]).map(lambda x: x.replace('ㆍ출력','').replace('․출력',''))
written_judges.loc[written_judges['총톤수'].map(lambda x: '기관종류' in x),'총톤수'] = written_judges.loc[written_judges['총톤수'].map(lambda x: '기관종류' in x),'총톤수'].map(lambda x: x.split('기관종류')[0])

In [ ]:
written_judges.to_csv('written_judges.csv')

### 사건 상세

In [ ]:
# 사건 상세 행 단어문서행렬로 만들기, 빈도 나타내기
cv2 = CountVectorizer(tokenizer=extract_n)
tdm2 = cv2.fit_transform(written_judges['사건상세'])
df2 = pd.DataFrame({'단어':cv2.get_feature_names(), '빈도':tdm2.sum(axis=0).flat})
df2.head()

In [ ]:
# 저장
df2.to_csv('wordcounts2.csv')
np.save('tdm2.npy', tdm2)
with open('cv2.pkl','wb') as f:
    pickle.dump(cv2,f)
# 불러오기
with open('cv2.pkl', 'rb') as f:
    cv2 = pickle.load(f)
tdm2 = np.load('tdm2.npy', allow_pickle=True).tolist()
df2 = pd.read_csv('wordcounts2.csv',index_col=0)

In [ ]:
n_lists = []
for text in tqdm(written_judges['사건상세'].dropna()): # 내용 형태소 분석
    n_list = []
    kiwi=Kiwi()
    kiwi.prepare()
    morpheme = kiwi.analyze(text)
    for lemma, pos, _, _ in morpheme[0][0]:
        if pos.startswith('N'): # 명사만 추출
            n_list.append(lemma)
    n_lists.append(n_list)

n_lists = pd.Series(n_lists) # 추출한 명사 합치기

ft2 = FastText(sg=1,sentences=n_lists) # FastText기법

In [ ]:
accidents = ['기관손상', '안전사고', '좌초', '부유물감김', '해양오염', '충돌', '전복', '화재 · 폭발', '침몰', '운항저해', '접촉']
similarities = []
for acc in combinations(accidents,2): # 사고 간 코사인 유사도 검사
    similarities.append((acc, ft2.wv.similarity(*acc)))
similarities

In [ ]:
sim_df = pd.DataFrame(similarities, columns=['acc','sim'])

In [ ]:
sim_df.to_csv('sim_df.csv')

In [ ]:
acc_sm = sim_df.set_index('acc').sort_values('sim',ascending=False) # 코사인 유사도를 기준으로 정렬

In [ ]:
acc_sm.plot.bar(figsize=(12,4), legend=False)
plt.title('사고 유형별 코사인 유사도')
plt.ylabel('cosine similarity')
plt.show()

In [ ]:
sm_lists = []
for i in accidents: # 단어 별 코사인 유사도 행렬 만들기
    sm_list = []
    for j in accidents:
        sm_list.append(ft2.wv.similarity(i,j))
    sm_lists.append(sm_list)

plt.figure(figsize=(8,6))
sns.heatmap(pd.DataFrame(columns=accidents,index=accidents,data=sm_lists), annot=True) # 히트맵 그리기
plt.title('사고 간 코사인 유사도')
plt.show()

### 사고발생원인

In [ ]:
# 사고발생원인 행 단어문서행렬로 만들기, 빈도분석
cv3 = CountVectorizer(tokenizer=extract_n)
tdm3 = cv3.fit_transform(written_judges['나.사고발생원인'].dropna())
df3 = pd.DataFrame({'단어':cv3.get_feature_names(), '빈도':tdm3.sum(axis=0).flat})
df3.head()

In [ ]:
# 저장
df3.to_csv('wordcounts3.csv')
np.save('tdm3.npy', tdm3)
with open('cv3.pkl','wb') as f:
    pickle.dump(cv3,f)
# 불러오기
with open('cv3.pkl', 'rb') as f:
    cv3 = pickle.load(f)
tdm3 = np.load('tdm3.npy', allow_pickle=True).tolist()
df3 = pd.read_csv('wordcounts3.csv',index_col=0)

In [ ]:
n_lists = []
for text in tqdm(written_judges['나.사고발생원인'].dropna()): # 내용 형태소 분석
    n_list = []
    kiwi=Kiwi()
    kiwi.prepare()
    morpheme = kiwi.analyze(text)
    for lemma, pos, _, _ in morpheme[0][0]:
        if pos=='NNG': # 일반 명사만 추출
            n_list.append(lemma)
    n_lists.append(n_list)

n_lists = pd.Series(n_lists)

ft3 = FastText(sg=1,sentences=n_lists) # FastText기법 적용

In [ ]:
ms_acc = []
for acc in accidents:
    ms_acc.append([acc,ft3.wv.most_similar(acc, topn=100)]) # 사고 별로 가장 많이 나오는 단어 100개 나타내기

In [ ]:
ms_df = pd.DataFrame(ms_acc, columns=['acc','ms_list'])

In [ ]:
ms_df['ms_list'] = ms_df.ms_list.map(lambda x: pd.DataFrame(x, columns=['word','sm']).set_index('word'))

In [ ]:
# barchart로 나타내기
fig, axes = plt.subplots(11,1,figsize=(20,44))
for i in tqdm(range(ms_df.shape[0])):
    ms_df.ms_list[i].plot.bar(ax=axes[i],legend=False, xlabel=False, color='gray')
    axes[i].set_title(f'{ms_df.acc[i]}')
plt.tight_layout()
plt.show()

In [ ]:
# 클러스터링
cl = DBSCAN(min_samples=2)
result_cl = pd.DataFrame(tuple(zip(ft2.wv.index_to_key,cl.fit_predict(ft2.wv.vectors))), columns=['word','cl'])

In [ ]:
for i in range(11):
    ms_df.ms_list[i] = pd.merge(ms_df.ms_list[i].reset_index(),result_cl,on='word',how='left')

In [ ]:
cl_count = []
for i in range(11):
    cl_count.append([ms_df.acc[i],ms_df.ms_list[i].cl.value_counts().shape[0]])

In [ ]:
pd.DataFrame(cl_count,columns=['acc','num_reason']).set_index('acc').plot.bar(xlabel=False,legend='', color='coral')
plt.title('사고 종류별 요인 개수')
plt.ylabel('요인 개수')
plt.grid()
plt.show()

In [ ]:
ms_df.to_csv('ms_df.csv')